<a href="https://www.berlin.de/en/"><img src = "https://www.deutschland.de/sites/default/files/styles/crop_page/public/media/image/berlin-ranking-most-expensive-city-Global-Residential-Cities-Index.jpg?itok=jMfalIkn" width = 700> </a>

<h1 align=center><font size = 6>Analysis of land prices and venues in Berlin</font></h1>

<h1 align=center><font size = 6>Week 1</font></h1>

## Background and problem description
---
Berlin, the capital of Germany with a total population of 3.6 Million and an area of around 890 km2 [1], has recently grown more and more as a popular city in Central Europe accompanied by "skyrocketing" property prices. A study by the property consultancy Knight Frank, mentioned in [2], attributed the increase in property prices to the population growth, the stable economy, relatively low unemployment rates and interests from investors.

Up to the year 2030 the population in Berlin will approximately grow by 4.7% or 177,000 inhabitants. The average age will rise only relatively gently because of the continuing immigration of young people from 42.7 years in 2018 to 42,9 years in 2030. The positive development of the natural changes of population as the ratio between birth-rate and death-rate due to the ongoing increase in birth-rates will remain stable because it is assumed that immigration of young people will continue. Up to the year 2025, it is assumed, that the birth-rate will exceed the death-rate depending on immigration dynamics [3].

As a consequence of the changes, the social and economic dynamics in the city will can be expected rise. The popularity of the city probably will also lead to a growth in tourism. For people with their businesses positioned in the tourism and cultural sector (city tours, trips, museums, restaurants etc.), the state and development of the city structure might be important. This includes the distribution of venues and their popularity in space and time. This knowledige is key to find popular neighbourhoods in order to place specific offers.

However, some (potential) negative effects for the inhabitants of the city include, e. g. increasing housing prices, increasing prices in food places and bars or a higher number of tourists potentially leading to discomfort for some people. Thus, for policy-makers an analysis of venue data might be interesting to understand the city and the population better.

Therefore, the following questions will be addressed in this project:
1. What are characteristic venues in certain places in the city of Berlin?
2. How are the characteristic venues related to housing or land prices?

## Data requirements and data description
---
In order to address the problems mentioned above, population, economic and venue data will be collected and analysed. This includes:

- borough geometry data (GeoJSON) [4],
- geographic positions of local centers [5],
- further information on boroughs i. e. population density [6] and land prices [7] and
- venue data of defined local centers [8].

The regional authority for statistics does not offer property prices by boroughs. Therefore, land prices will be used as a proxy for the popularity of a borough. In total, there are 12 boroughs in Berlin. In order to assess the venues in more detail, local centers will be defined and will be based on the zip codes. In total, Berlin has 190 zip codes distributed within the city area.

## Methodology
---
### Data collection, data wrangling and data cleaning
The main Python modules, that will be used for working with the data include:

- Request,
- BeautifulSoup,
- Pandas,
- Numpy,
- Geocoder and
- the Foursquare API.

They will be used for collecting and cleaning the data, reorganizing and transforming data including One-Hot-Encoding.

### Clustering: K-Means
For further data analysis and modelling, K-Means, a non-supervised machine learning algorithm, will be used for clustering the venue data. Euclidian distance will be used as a metric for the distance between individual venue data points and the centroids defining the clusters. An important unknown will be the number of clusters, that has to be defined beforehand. As the total error (distances of data points to centroids) will always decrease with an increasing number of clusters, the "Elbow method" will be used in order to find the optimal k. The "elbow point" is defined by a sudden softer decrease of the total error with increasing numbers of clusters.

### Data visualization
Data visualization is important for understanding (spatial) relationships between data results and communicating these results in order to reach clear conclusions of the analyses. Matplotlib and Folium will be used as the primary visualization tools for

1) understand the data and

2) link model results to other spatial features.

by generating bar plots of cluster categories or by creating overview maps of the spatial distribution of the clusters and choropleth maps of land prices.

### Data example
An example will be given on how data will be retrieved and processed, e. g. population density of districts.

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#scrape population density 2019 statistics from website and transform to BS-object
url = 'https://de.wikipedia.org/wiki/Verwaltungsgliederung_Berlins'
website_content = requests.get(url).text
soup = BeautifulSoup(website_content,'lxml')

#extract land price table from html
table = soup.find('table',{'class':'wikitable sortable zebra'})
df2 = pd.read_html(str(table), decimal=',', thousands='.')[0]

#cut land table and drop unnecessary data
df2 = df2.iloc[:,[1,5]] #drop rows

#change columns' names
cols = ['borough name','population density 1000 by km2']
df2.columns = cols

#rescale values
df2['population density 1000 by km2']=df2['population density 1000 by km2']/1000.

df2

,borough name,population density 1000 by km2
0,Mitte,9.733
1,Friedrichshain-Kreuzberg,14.373
2,Pankow,3.958
3,Charlottenburg-Wilmersdorf,5.289
4,Spandau,2.655
5,Steglitz-Zehlendorf,3.012
6,Tempelhof-Schöneberg,6.624
7,Neukölln,7.338
8,Treptow-Köpenick,1.610
9,Marzahn-Hellersdorf,4.350


Furthermore, a first overview of the boroughs, distribution of local centers and the population density will be given.

In [7]:
local_centers = pd.read_csv('http://www.fa-technik.adfc.de/code/opengeodb/PLZ.tab', sep='\t')

#Select Berlin neighbourhoods from Germany
local_centers = local_centers[local_centers['Ort']=='Berlin']

#select only zip code and coordinates
local_centers = local_centers[['plz','lat', 'lon',]]

#make zip code the name
local_centers = local_centers.rename(columns={'plz': 'name'})

local_centers

,name,lat,lon
686,10115,52.533707,13.387224
687,10117,52.518746,13.390193
688,10119,52.532666,13.407149
689,10178,52.523474,13.412203
690,10179,52.514591,13.419699
...,...,...,...
873,14169,52.446430,13.265875
874,14193,52.476342,13.232888
875,14195,52.459317,13.286944
876,14197,52.475009,13.314679


In [13]:
import geocoder
import folium
from IPython.display import HTML, display

berlin = geocoder.osm('Berlin, DE')
berlin.latlng

#supply local GeoJSON file path
berlin_geojson='/Users/Alex/Downloads/Berlin-Geodaten-master/berlin_bezirke.geojson'

# create a plain map of Berlin
map_berlin = folium.Map(location=berlin.latlng, zoom_start=10, tiles='cartodbpositron')

#define thresholds
threshold_scale = [1,2,3,5,7.5,10,15]

# generate choropleth map using the land prices from 2018 of each borough
map_berlin.choropleth(
    geo_data=berlin_geojson,
    data=df2,
    columns=['borough name', 'population density 1000 by km2'],
    key_on='feature.properties.name',
    threshold_scale=threshold_scale,
    fill_color='YlGnBu', 
    fill_opacity=0.6, 
    line_opacity=0.3,
    legend_name='Population Density [Inhabitants/km2]')

for lat, lon, name in zip(local_centers.lat, local_centers.lon, local_centers.name):
    folium.CircleMarker(
        [lat, lon],
        radius=4, # define how big you want the circle markers to be
        color='k',
        fill=True,
        popup=name,
        fill_color='k',
        fill_opacity=0.6
    ).add_to(map_berlin)
    
display(map_berlin)

## Sources
---
[1] Wikipedia article: https://de.wikipedia.org/wiki/Berlin (accessed: 17 March 2020)

[2] deutschland.de (2018): https://www.deutschland.de/en/topic/life/berlin-property-price-growth-fastest-in-world (accessed: 17 March 2020)

[3] SenStadtWohn (2020): https://www.stadtentwicklung.berlin.de/planen/bevoelkerungsprognose/de/ergebnisse/index.shtml (accessed: 17 March 2020)

[4] Berlin GeoJSON file: https://github.com/funkeinteraktiv/Berlin-Geodaten (accessed: 17 March 2020)

[5] ADFC Fachabteilung Technik (zip codes): 'http://www.fa-technik.adfc.de/code/opengeodb/PLZ.tab' (accessed: 17 March 2020)

[5] Wikipedia data table: https://de.wikipedia.org/wiki/Verwaltungsgliederung_Berlins  (accessed: 17 March 2020)

[6] Amt für Statistik Berlin und Brandenburg: https://www.statistik-berlin-brandenburg.de/regionalstatistiken/r-gesamt_neu.asp?Ptyp=410&Sageb=61005&creg=BBB&anzwer=6 (accessed: 17 March 2020)

[7] Foursquare API venue data: https://de.foursquare.com (accessed: 17 March 2020)